In [ ]:
## import os and system packages
import os, sys

## add acolite to path and import
sys.path.append('acolite')
import acolite as ac

In [ ]:
## add proj lib env variable to avoid OGR Error: Corrupt data
## should not be needed, but occurs in certain set ups (like Copernicus JupyterHub)
os.environ['PROJ_LIB'] = "/opt/conda/envs/acolite/share/proj/"

In [ ]:
## CDSE credentials are needed for retrieving data
## either add CDSE_u and CDSE_p credentials to credentials.txt
## CDSE_u= 
## CDSE_p= 
## or add them directly in environment variables
## os.environ['CDSE_u'] = ''
## os.environ['CDSE_p'] = ''

## load credentials and set as env variables
credentials = ac.shared.import_config('credentials.txt')
for k in credentials: os.environ[k] = credentials[k]

In [ ]:
## location and date of interest
## you can use the Copernicus browser to find images
## https://browser.dataspace.copernicus.eu/
region_name = 'Spuikom'
station_lon = 2.9522
station_lat = 51.2278
date = '2025-08-25'

In [ ]:
## use ACOLITE API to query CDSE
urls, scenes = ac.api.cdse.query(roi = 'POINT ({} {})'.format(station_lon, station_lat),
                                 collection = "SENTINEL-2", start_date = date, end_date = date)
## print the results
for scene in scenes:
  print(scene)

In [ ]:
## use ACOLITE API to download the scenes from CDSE
local_scenes = ac.api.cdse.download(urls, scenes = scenes, output = 'Input/')

In [ ]:
# acolite settings
settings = {## basic input and output settings
            ## local_scenes list is used as returned from the API download function
            'inputfile': local_scenes,
            'output': 'Output/',

            ## set up region of interest centred on lon/lat, with box size in km
            'region_name': region_name,
            'station_lon': station_lon,
            'station_lat': station_lat,
            'station_box_size': 7,
            'station_box_units': 'km',

            ## disable ancillary data retrieval, set to True if EarthData credentials are set
            'ancillary_data': False,

            ## use RAdCor processing
            'atmospheric_correction_method': 'radcor',

            ## add a buffer and crop to centre to avoid masked data around the scene edges
            'limit_buffer': 5,
            'limit_buffer_units': 'km',
            'radcor_crop_centre': True,

            ## add the following to enable SWIR based glint correction
            # 'dsf_residual_glint_correction': True,
            # 'glint_mask_rhos_threshold': 0.10,

            ## delete L1R output NetCDF
            'l1r_delete_netcdf': True,
            ## don't output L1R RGBs
            'l1r_rgb_keys': [],
            ## delete ACOLITE text outputs
            'delete_acolite_run_text_files': True,
            }

In [ ]:
## run acolite with settings dict
ret = ac.acolite.acolite_run(settings)